In [1]:
import os
import torch


In [2]:
# Change the working directory to the parent directory
os.chdir("..")

# Verify the current working directory
print("Current Directory:", os.getcwd())

Current Directory: d:\workspace\iscat


In [3]:
from src.data_processing.dataset import iScatDataset
from src.data_processing.utils import Utils
data_path = os.path.join('data', 'iScat', 'Data', '2024_11_11', 'Metasurface', 'Chip_02')
image_paths,target_paths = Utils.get_data_paths(data_path)
dataset = iScatDataset(image_paths, target_paths, preload_image=True)

ImageJ2 version: 2.14.0/1.54f


Creating Masks: 100%|██████████| 5/5 [11:01<00:00, 132.25s/it]


(5, 2160, 2560)
(500, 2160, 2560)


In [18]:
torch.all(dataset.masks[0]==dataset.masks[101])

tensor(False)

In [5]:
image_paths[2]

'data\\iScat\\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 03\\Brightfield\\02_03_BF.nd2'